In [2]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
from sklearn.metrics import r2_score
import pickle
from tqdm import tqdm
import csv

#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

#Import decoder functions
from Neural_Decoding.decoders import WienerFilterDecoder


#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

#Import hyperparameter optimization packages
try:
    from hyperopt import fmin, hp, Trials, tpe, STATUS_OK
except ImportError:
    print("\nWARNING: hyperopt package is not installed. You will be unable to use section 5.")
    pass

#Import decoder functions
from Neural_Decoding.decoders import LSTMDecoder

In [7]:
# import the data
import os

path = '../../preprocessed_data/shifted_ind_tar/'

R2 = np.empty((2,8))
#LSTM_models = []

col = 0
for filename in os.listdir(path):
    if filename.endswith('.pickle'):
        with open(os.path.join(path, filename),'rb') as f:
            M1, PMd, M1_PMd,pos_binned,vels_binned=pickle.load(f,encoding='latin1') #If using python 3
            
            # group neural data and kinematics
            neural_data_combined = [M1]
            kinematics = [pos_binned, vels_binned]
            for (i, neural_data) in enumerate(neural_data_combined):
                for (j, output) in enumerate(kinematics):
                    # Preprocess data
                    bins_before=6 #How many bins of neural data prior to the output are used for decoding
                    bins_current=1 #Whether to use concurrent time bin of neural data
                    bins_after=0 #How many bins of neural data after the output are used for decoding

                    # Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
                    # Function to get the covariate matrix that includes spike history from previous bins
                    X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

                    # Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
                    #Put in "flat" format, so each "neuron / time" is a single feature
                    X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))
                
                    # Output covariates
                    #Set decoding output
                    y=output

                    # Split into training / testing / validation sets
                    #Set what part of data should be part of the training/testing/validation sets
                    training_range=[0, 0.7]
                    testing_range=[0.7, 0.85]
                    valid_range=[0.85,1]

                    # Split data:
                    num_examples=X.shape[0]

                    #Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
                    #This makes it so that the different sets don't include overlapping neural data
                    training_set=np.arange(int(np.round(training_range[0]*num_examples))+bins_before,int(np.round(training_range[1]*num_examples))-bins_after)
                    testing_set=np.arange(int(np.round(testing_range[0]*num_examples))+bins_before,int(np.round(testing_range[1]*num_examples))-bins_after)
                    valid_set=np.arange(int(np.round(valid_range[0]*num_examples))+bins_before,int(np.round(valid_range[1]*num_examples))-bins_after)

                    #Get training data
                    X_train=X[training_set,:,:]
                    X_flat_train=X_flat[training_set,:]
                    y_train=y[training_set,:]

                    #Get testing data
                    X_test=X[testing_set,:,:]
                    X_flat_test=X_flat[testing_set,:]
                    y_test=y[testing_set,:]

                    #Get validation data
                    X_valid=X[valid_set,:,:]
                    X_flat_valid=X_flat[valid_set,:]
                    y_valid=y[valid_set,:]

                    # Process covariates
                    #Z-score "X" inputs. 
                    #X_train_mean=np.nanmean(X_train,axis=0)
                    #X_train_std=np.nanstd(X_train,axis=0)
                    #X_train=(X_train-X_train_mean)/X_train_std
                    #X_test=(X_test-X_train_mean)/X_train_std
                    #X_valid=(X_valid-X_train_mean)/X_train_std

                    #Z-score "X_flat" inputs. 
                    #X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
                    #X_flat_train_std=np.nanstd(X_flat_train,axis=0)
                    #X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
                    #X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std
                    #X_flat_valid=(X_flat_valid-X_flat_train_mean)/X_flat_train_std

                    #Zero-center outputs
                    #y_train_mean=np.mean(y_train,axis=0)
                    #y_train=y_train-y_train_mean
                    #y_test=y_test-y_train_mean
                    #y_valid=y_valid-y_train_mean

                    #Do optimization
                    # Define parameters for hyperoptimisation
                    def lstm_evaluate2(params):
                        #Put parameters in proper format
                        num_units=int(params['num_units'])
                        frac_dropout=float(params['frac_dropout'])
                        n_epochs=int(params['n_epochs'])
                        model_lstm=LSTMDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs) #Define model
                        model_lstm.fit(X_train,y_train) #Fit model
                        y_valid_predicted_lstm=model_lstm.predict(X_valid) #Get validation set predictions
                        #del model_lstm
                        # K.clear_session()
                        #gc.collect()
                        return -np.mean(get_R2(y_valid,y_valid_predicted_lstm)) #Return -R2 value of validation set

                    ##The range of values I'll look at for the parameter
                    #"hp.quniform" will allow us to look at integer (rather than continuously spaced) values.
                    #So for "num_units", we are looking at values between 50 and 700 by 10 (50,60,70,...700)
                    #"hp.uniform" looks at continuously spaced values
                    space = {
                        'frac_dropout': hp.uniform('frac_dropout', 0., 0.5),
                        'num_units': hp.quniform('num_units', 50,700,10),
                        'n_epochs': hp.quniform('n_epochs', 2,15,1),
                    }

                    #object that holds iteration results
                    trials = Trials()

                    #Set the number of evaluations below
                    hyperoptBest = fmin(lstm_evaluate2, space, algo=tpe.suggest, max_evals=5, trials=trials)

                    #Declare model
                    model_lstm=LSTMDecoder(units=int(hyperoptBest['num_units']),dropout=hyperoptBest['frac_dropout'],num_epochs=int(hyperoptBest['n_epochs']))

                    #Fit model
                    model_lstm.fit(X_train,y_train)

                    #Get predictions
                    y_valid_predicted_lstm=model_lstm.predict(X_valid)

                    R2_vw = r2_score(y_valid,y_valid_predicted_lstm, multioutput='variance_weighted')

                    # Save the R2 value for a given neural data and kinematics
                    R2[2*i+j,col] = R2_vw
            col = col +1
                    
                    # store the model - maybe needed later
                    #LSTM_models.append(model_lstm)
            

100%|█████████████████████████████████████████████████| 5/5 [01:47<00:00, 21.53s/trial, best loss: -0.6349182545240578]


In [5]:
# get average of R2 values for each kinematic
R2_mean = np.average(R2,axis=1)
R2_std = np.std(R2,axis=1)

# Export the data into a csv file for further analysis
csv_rowlist = [['R^2 values','LSTM', 'M1 data'], ['Target','1','2','3','4','5','6','7','8','Average', 'Std. dev.'], ['position',R2[0,0],R2[0,1],R2[0,2],R2[0,3],R2[0,4],R2[0,5],R2[0,6],R2[0,7],R2_mean[0],R2_std[0]], ['velocity',R2[1,0],R2[1,1],R2[1,2],R2[1,3],R2[1,4],R2[1,5],R2[1,6],R2[1,7],R2_mean[1],R2_std[1]]]

export_path = '../Results/Ind_tar/LSTM_raw_RS_tar.csv'
with open(export_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_rowlist)

In [4]:
print(R2)

[[0.41878702 0.52425775 0.3327082  0.28673085 0.3037638  0.52413294
  0.41643759 0.45529486]
 [0.65737605 0.56719976 0.68310779 0.59716969 0.61155576 0.50511779
  0.41563277 0.71745977]]


In [8]:
R2_mean = np.average(R2,axis=1)
R2_std = np.std(R2,axis=1)
print(R2_mean)
print(R2)

[0.35412392 0.56015296]
[[0.33547591 0.33499651 0.52954367 0.12061134 0.35266716 0.47558614
  0.26822239 0.41588823]
 [0.5760749  0.5679306  0.69156854 0.59056518 0.66014882 0.43365187
  0.34942368 0.61186009]]
